# 🧩 Lab 4 – Structured Story Generation with Chaining

In this notebook, you'll break your story into multiple parts and generate it step by step using a large language model.

Instead of generating the full story in one go, you'll first ask the model to create a table of contents, and then generate each chapter separately. This teaches you how to **call the model multiple times** and **use its output as input** for the next step.

You'll need to:
- Write your own creative story idea
- Parse the model's output into clean chapter summaries
- Accumulate the story as you go

By the end, you'll have a **multi-part story** with coherent structure and flow.

In [ ]:
from IPython.display import display, Markdown
import ollama

# 🔧 Configuration
MODEL_NAME = "gemma3:4b-it-qat"  # You can try larger models if GPU allows
NUM_PARTS = 5  # How many chapters should the story have

# TODO: Write your own story idea below! Be creative.
story_idea = ""

assert story_idea != ""


# STEP 1: Generate abstracts / table of content

system_prompt_toc = f"""
You are a storyteller. We're going to generate a multi-part story.
Your task is to plan the structure, by generating story abstracts.

Given the user prompt as idea, generate {NUM_PARTS} one-line abstracts.
Each abstract should represent the content of one part of the story.
Do not write the full story yet. Just output the abstracts, one per line.
Only output the {NUM_PARTS} one-line abstracts. Nothing else.
No extra explanations. No empty lines. No chit chat.
"""

# Generate the list of abstracts
response_toc = ollama.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": system_prompt_toc},
        {"role": "user", "content": story_idea}
    ]
)

abstracts_raw = response_toc['message']['content'].strip()

# TODO: Turn the raw response into a list of clean one-line abstracts
# Make sure to remove empty lines and extra whitespace.
# Hint: Use .split("\n") to break lines, then .strip() to remove whitespace.
abstracts = []  # ← You'll need to extract clean lines from `abstracts_raw`

assert len(abstracts) != 0

abstracts_full = "\n".join(abstracts)

print("=== Abstracts ===")
print(abstracts_full)

# STEP 2: Generate story parts

full_story_so_far = ""

for i, abstract in enumerate(abstracts):
    system_prompt_chapter = f"""
You are a storyteller. We're building a multi-part story.
Please write the next part of the story.
Use previous parts as context to ensure continuity.
Make sure the text flows naturally from earlier events.
"""

    user_prompt_chapter = f"""
All abstracts for the full story (one per line):
{abstracts_full}

Here is the story so far:
{full_story_so_far}

Now continue with part {i+1}: {abstract}
Each part should be no more than 2 paragraphs!
"""

    response_chapter = ollama.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt_chapter},
            {"role": "user", "content": user_prompt_chapter}
        ]
    )

    chapter_text = response_chapter["message"]["content"].strip()

    # TODO: Accumulate the story so far by appending this chapter
    # (Hint: use += to add the current chapter text)
    # full_story_so_far = ...

    assert full_story_so_far != ""

    # STEP 3: Display the Final Story
    display(Markdown(f"### Part {i+1}: {abstract}\n\n"))
    display(Markdown(chapter_text))